In [1]:
# [Cell 1: .env 로드 및 Agent 임포트]
# .env 파일 로드 (core.py가 로드하지만, 노트북 환경을 위해 한 번 더)
import sys
import os
from dotenv import load_dotenv

# 1. 현재 프로젝트 루트 디렉토리(jesafe/)의 절대 경로를 찾습니다.
# os.getcwd() (현재 폴더)의 부모 폴더('..')가 프로젝트 루트라고 지정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# 2. 이 경로를 파이썬 모듈 검색 경로(sys.path)에 추가합니다.
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"프로젝트 루트 경로 추가: {project_root}")

load_dotenv()
print(".env 로드 완료:", os.getenv("DATABASE_URL") is not None)

프로젝트 루트 경로 추가: /Users/lwj0831/jesafe
.env 로드 완료: True


In [9]:
# [Cell 2: 메인 Agent 동기 테스트 (RunnableWithMessageHistory 사용)]
from src.services import main_agent_executor
from src.data_loader import _load_jeju_coords

# 테스트 전에 CSV 데이터를 메모리에 로드합니다.
_load_jeju_coords()

# 테스트에 사용할 고유 세션 ID
session_id = "test_session_notebook_123"

# --- 1. 날씨 질문 테스트 ---
# (프롬프트 규칙에 따라 '제주도'가 '제주시'로 변환되어야 함)
query1 = "오늘 제주도 날씨 어때?"
print(f"User: {query1}")

response1 = main_agent_executor.invoke(
    {"input": query1},
    config={"configurable": {"session_id": session_id}}
)
print(f"AI: {response1['output']}\n")
# (agent_with_history가 이 대화를 session_id에 자동 저장)


# --- 2. 사고 통계 질문 테스트 ---
query2 = "제주시에서 낙상 사고가 몇 건이야?"
print(f"User: {query2}")

response2 = main_agent_executor.invoke(
    {"input": query2},
    config={"configurable": {"session_id": session_id}}
)
print(f"AI: {response2['output']}\n")
# (agent_with_history가 이 대화를 session_id에 자동 저장)


# --- 3. (중요) 대화 맥락(Context) 질문 테스트 ---
# (이전 질문의 '제주시'를 기억하고 '제주시' 날씨를 답해야 함)
query3 = "그럼 그 지역 날씨는 어때?"
print(f"User: {query3}")

response3 = main_agent_executor.invoke(
    {"input": query3},
    config={"configurable": {"session_id": session_id}}
)
print(f"AI: {response3['output']}\n")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


✅ [data_loader] 제주도 (KMA 격자) DB 로드 완료 (경로: /Users/lwj0831/jesafe/data/kma_jeju_grid_info.csv)
User: 오늘 제주도 날씨 어때?


KeyError: "Input to ChatPromptTemplate is missing variables {'chat_history'}.  Expected: ['agent_scratchpad', 'chat_history', 'input'] Received: ['input', 'intermediate_steps', 'agent_scratchpad']\nNote: if you intended {chat_history} to be part of the string and not a variable, please escape it with double curly braces like: '{{chat_history}}'."

In [8]:
# [Cell 3: RAG 임베딩 및 리트리버 전체 테스트]
import sys
import os

# (Cell 1과 동일) 프로젝트 루트 경로 설정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.core import get_vector_store, get_bm25_retriever
from src.core import _compression_retriever_instance

print("Connecting to Vector Store & Initializing Retrievers...")
try:
    # 앱 시작 시점과 동일하게 리소스 초기화
    vector_store = get_vector_store()
    bm25_retriever = get_bm25_retriever()
    
    # 테스트 쿼리
    query = "제주도 오름 추천해줘"
    print(f"\nTest Query: '{query}'")
    
    # -----------------------------------------------
    # 테스트 1: PGVector (의미기반 검색)
    # -----------------------------------------------
    print("\n--- 1. PGVector (Similarity Search) Test ---")
    vector_results = vector_store.similarity_search(query, k=3)
    print(f"Found {len(vector_results)} Chunks:")
    for i, doc in enumerate(vector_results):
        print(f"  [Vec {i+1}] {doc.page_content[:100]}...")

    # -----------------------------------------------
    # 테스트 2: BM25 (키워드 기반 검색)
    # -----------------------------------------------
    print("\n--- 2. BM25 (Keyword Search) Test ---")
    bm25_results = bm25_retriever.invoke(query) 
    print(f"Found {len(bm25_results)} Chunks:")
    for i, doc in enumerate(bm25_results):
        print(f"  [BM25 {i+1}] {doc.page_content[:100]}...")

    # -----------------------------------------------
    # 테스트 3: Ensemble (최종 앙상블)
    # -----------------------------------------------
    print("\n--- 3. Ensemble Retriever Test ---")
    ensemble_results = _compression_retriever_instance.base_retriever.invoke(query)
    print(f"Found {len(ensemble_results)} Chunks (after re-ranking):")
    for i, doc in enumerate(ensemble_results):
        print(f"  [Ensemble {i+1}] (Source: {doc.metadata.get('source', 'N/A')})")
        print(f"    {doc.page_content[:200]}...")

except Exception as e:
    print(f"\n❌ 테스트 실패: {e}")

Connecting to Vector Store & Initializing Retrievers...

Test Query: '제주도 오름 추천해줘'

--- 1. PGVector (Similarity Search) Test ---
Found 3 Chunks:
  [Vec 1] 별도봉은 제주시의 전경과 제주항을 오고가는 배들의 모습으로 볼 수 있는 곳으로 연인끼리 혹은 가족끼리 편하게 산책할 수 있는 아름다운 장소다.

제주시 화북1동 4472 탐방시간 ...
  [Vec 2] 제주시 구좌읍 종달리 산70 탐방시간 : 2시간 주변여행 : 제주자연생태공원, 비자림, 청초밭

복합형

큰노꼬메오름

난이도상 #한라산 조망 #넓은 주차장 #2개의 분화구

오름...
  [Vec 3] 긴 소매 옷, 긴 바지는 필수!

승마 너른 벌판을 달려라

넓은 초원을 달리고 싶다면 제주에서의 승마를

추천한다. 말위에서 느끼는 제주의 바람은 또 다른 맛을

선사해 준다.
...

--- 2. BM25 (Keyword Search) Test ---
Found 3 Chunks:
  [BM25 1] Ver 1.1

제주도 공식 관광정보

VISIT JEJU

4

CONTENTS

오름이란?……………………………………6

오름 트레킹 가이드… ……………………7

초보자를 위한 ...
  [BM25 2] 오름 트레킹 문의 및 정보

제주관광정보센터 064-740-6000 www.visitjeju.net

제주특별자치도 오름 탐방 064-120 https://gis.jeju.go.k...
  [BM25 3] 별도봉은 제주시의 전경과 제주항을 오고가는 배들의 모습으로 볼 수 있는 곳으로 연인끼리 혹은 가족끼리 편하게 산책할 수 있는 아름다운 장소다.

제주시 화북1동 4472 탐방시간 ...

--- 3. Ensemble Retriever Test ---
Found 5 Chunks (after re-ranking):
  [Ensemble 1] (Source: data/tourism_docs/

In [7]:
# [Cell 4: EmbeddingsFilter 상세 디버깅]
import sys
import os
import numpy as np
from langchain_community.utils.math import cosine_similarity

# 1. (Cell 1과 동일) 프로젝트 루트 경로 설정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# 2. src/core.py에서 핵심 컴포넌트 임포트
from src.core import get_compression_retriever, get_cached_embedder

# 3. 테스트 쿼리 정의
query = "제주에서 갈만한 맛집 추천해줘"
similarity_threshold = 0.85 # ⬅️ core.py에 설정된 임계값

print(f"Query: {query}")
print(f"Similarity Threshold: {similarity_threshold}")
print("-" * 30)

try:
    # 4. 리트리버 및 임베더 로드
    compression_retriever = get_compression_retriever()
    # 4-1. 압축 전 '기본' 리트리버 (EnsembleRetriever)
    base_retriever = compression_retriever.base_retriever
    # 4-2. 점수 계산에 사용할 임베더
    embedder = get_cached_embedder()

    # 5. (압축 전) 앙상블 리트리버가 찾은 문서 원본 가져오기
    print("Fetching docs from EnsembleRetriever (before filtering)...")
    retrieved_docs = base_retriever.invoke(query)
    
    if not retrieved_docs:
        print("EnsembleRetriever가 문서를 찾지 못했습니다.")
    else:
        print(f"Found {len(retrieved_docs)} docs. Calculating similarity scores...")
        
        # 6. (핵심) 쿼리와 각 문서의 임베딩 점수 수동 계산
        query_embedding = embedder.embed_query(query)
        doc_embeddings = embedder.embed_documents([doc.page_content for doc in retrieved_docs])
        
        # 7. 코사인 유사도 계산
        scores = cosine_similarity([query_embedding], doc_embeddings)[0]
        
        # 8. 결과 출력
        for i, (doc, score) in enumerate(zip(retrieved_docs, scores)):
            is_passed = score >= similarity_threshold
            
            print(f"\n[Doc {i+1}] Score: {score:.4f} (Threshold: {similarity_threshold})")
            print(f"  -> PASSED: {is_passed} ⬅️")
            print(f"  -> Content: {doc.page_content[:150]}...")

except Exception as e:
    print(f"\n❌ 테스트 실패: {e}")

Query: 제주에서 갈만한 맛집 추천해줘
Similarity Threshold: 0.85
------------------------------
Fetching docs from EnsembleRetriever (before filtering)...
Found 6 docs. Calculating similarity scores...

[Doc 1] Score: 0.8639 (Threshold: 0.85)
  -> PASSED: True ⬅️
  -> Content: 지역색이 강해 향토음식이 다소 버거운 여행자라면 걱정하지 말기를.

지금 제주는 전국 어디서도 뒤지지 않는 맛집들이 풍부한 섬이다.

싱싱한 로컬푸드를 활용하고 참신한 아이디어까지 더한 핫한 음식들이 여기에 있다.

두루치기

고추장 양념된 돼지고기에 콩나물, 파채 등...

[Doc 2] Score: 0.8513 (Threshold: 0.85)
  -> PASSED: True ⬅️
  -> Content: 회를 떠서 비린맛이 전혀

없고 입안을 감도는 기름기의

고소함에 놀라게 된다. 부추

등 향채소가 가득한 매콤한

양념장과 생김을 곁들이면

뒷맛을 깔끔하게 만들어

제주에서의 특별한 회의

맛을 느껴볼 수 있다.

꼭 맛봐야 할 고등어 요리

고등어조림

제주에서는...

[Doc 3] Score: 0.8506 (Threshold: 0.85)
  -> PASSED: True ⬅️
  -> Content: 꿩의 가슴살을 얇게 저며 샤부샤부로 먹는

꿩토렴은 별미이다. 제주산 메밀과 꿩 육수가

어우러진 꿩메밀국수와 꿩만두국은 기름기

없이 깔끔한 맛을 자랑한다.

성게미역국

성게를 넣고 끓인 성게미역국은 제주도 경조사에 올라오는 단골 음식이다. 순두부처럼 엉겨

달짝지...

[Doc 4] Score: 0.8368 (Threshold: 0.85)
  -> PASSED: False ⬅️
  -> Content: 심심찮게 볼 수 있다. 천연발효종을 이용한 빵. 제주에서 나는

당근을 이용한 당

In [4]:
# [Cell 5: DDGS 웹 검색 결과 수동 테스트]
import sys
import os

# 1. (Cell 1과 동일) 프로젝트 루트 경로 설정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# 2. ddgs 라이브러리 직접 임포트
try:
    from ddgs import DDGS
except ImportError:
    print("❌ 'ddgs' 패키지가 설치되지 않았습니다. 'poetry add ddgs'를 실행하세요.")

# 3. 테스트 쿼리
query = "제주도 7월 축제"

print(f"Testing DDGS() with query: '{query}'")
print(f"Requesting max_results=3...")
print("-" * 30)

try:
    # 4. src/tools.py의 핵심 로직 직접 실행
    # (DDGS()는 컨텍스트 매니저(with) 사용을 권장)
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=3)
    
    # 5. 결과 확인
    if not results:
        print("❌ 검색 결과가 없습니다.")
    else:
        print(f"✅ {len(results)}개의 결과를 성공적으로 가져왔습니다:")
        
        for i, res in enumerate(results):
            print(f"\n[Result {i+1}]")
            print(f"  Title: {res.get('title')}")
            print(f"  Snippet: {res.get('body', '')[:150]}...")
            print(f"  URL: {res.get('href')}")

except Exception as e:
    print(f"❌ 테스트 실패: {e}")

Testing DDGS() with query: '제주도 7월 축제'
Requesting max_results=3...
------------------------------
✅ 3개의 결과를 성공적으로 가져왔습니다:

[Result 1]
  Title: 제주시, 제주도 , 대한민국 월별 날씨 | AccuWeather
  Snippet: Get the monthly weather forecast for 제주시, 제주도 , 대한민국, including daily high/low, historical averages, to help you plan ahead....
  URL: https://www.accuweather.com/ko/kr/jeju/224209/july-weather/224209

[Result 2]
  Title: 제주도 11 월 축제 한눈에 보기! | Brunch Story
  Snippet: 태진아와 워너원, AOA, 레드벨벳, EXID, 오마이걸, 모모랜드, 나인뮤지스, 여자친구 등 총 28팀이 참가하는 제주도 최대 K팝 행사로 기대를 모으고 있답니다. 제주종합경기장 주경기장. 제주특별자치도 제주시 서광로2길 24....
  URL: https://brunch.co.kr/@gorrajeju/298

[Result 3]
  Title: 중3 수학여행 장기자랑 제주도 서연 | TikTok
  Snippet: #수학여행 #08 #07 # 제주도 #ootd #fyp 제주도 수학여행 코디 아이디어와 팁. 여름 제주도 수학여행을 위한 스타일과 코디를 소개합니다....
  URL: https://www.tiktok.com/discover/중3-수학여행-장기자랑-제주도-서연
